In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from time import perf_counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from IPython.display import Markdown, display
from tensorflow import keras
from tensorflow.keras import layers
import cv2 as cv
import ShapePreprocess
import glob

2022-11-21 21:57:42.707914: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-21 21:57:43.155080: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-21 21:57:44.560461: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ohpc/pub/apps/cuda/11.3/lib64
2022-11-21 21:57:44.560642: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.s

## Load Data

In [8]:
data = np.load('data.npz')
x = data['x'] / 255
y = data['y']
num_classes = len(np.unique(y))
y = keras.utils.to_categorical(y, num_classes)
print(x.shape)
print(y.shape)

(46640, 150, 150)
(46640, 11)


In [9]:
print("x_train shape:", x.shape)
print("y_train shape:", y.shape)
print("class number:", num_classes)

x_train shape: (46640, 150, 150)
y_train shape: (46640, 11)
class number: 11


## Modeling (Simple CNN)

In [5]:
input_shape = (150, 150, 1)

In [6]:
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape))

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

# Compiling the CNN
cnn.compile(optimizer = 'adam',
            loss = 'categorical_crossentropy',
            metrics = ['accuracy'])
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   


2022-11-21 21:57:52.551652: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-21 21:57:57.594312: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10405 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1
2022-11-21 21:57:57.601955: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10405 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1
2022-11-21 21:57:57.607457: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/t

 conv2d (Conv2D)             (None, 148, 148, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 41472)             0         
                                                                 
 dense (Dense)               (None, 128)               5308544   
                                                                 
 dense_1 (Dense)             (None, 11)                1419      
          

## Data Argumentation

In [10]:
x = x.reshape(46640,150,150,1)
data_generator = ImageDataGenerator(
    rotation_range=180
)
data_generator.fit(x)

## Model Training

In [11]:
# 학습/검증 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=0)

print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

41976 train samples
4664 test samples


In [12]:
epochs = 30

cnn.fit_generator(data_generator.flow(x, y, batch_size=32),
                  validation_data=(x,y), 
                  steps_per_epoch=len(x) / 32,
                  epochs=epochs)

/tmp/ipykernel_159581/4124846619.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  cnn.fit_generator(data_generator.flow(x, y, batch_size=32),


Epoch 1/30


2022-11-21 22:05:21.809180: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201


1457/1457 [==============================] - 198s 131ms/step - loss: 0.3460 - accuracy: 0.8812 - val_loss: 0.2408 - val_accuracy: 0.9264
Epoch 2/30
1457/1457 [==============================] - 163s 112ms/step - loss: 0.2157 - accuracy: 0.9269 - val_loss: 0.2258 - val_accuracy: 0.9394
Epoch 3/30
1457/1457 [==============================] - 167s 115ms/step - loss: 0.1779 - accuracy: 0.9384 - val_loss: 0.2133 - val_accuracy: 0.9443
Epoch 4/30
1457/1457 [==============================] - 160s 110ms/step - loss: 0.1565 - accuracy: 0.9455 - val_loss: 0.1734 - val_accuracy: 0.9452
Epoch 5/30
1457/1457 [==============================] - 168s 115ms/step - loss: 0.1476 - accuracy: 0.9479 - val_loss: 0.1969 - val_accuracy: 0.9452
Epoch 6/30
1457/1457 [==============================] - 160s 110ms/step - loss: 0.1395 - accuracy: 0.9510 - val_loss: 0.1827 - val_accuracy: 0.9500
Epoch 7/30
1457/1457 [==============================] - 162s 111ms/step - loss: 0.1353 - accuracy: 0.9517 - val_loss: 0.156

KeyboardInterrupt: 

In [14]:
score = cnn.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.12110355496406555
Test accuracy: 0.9603344798088074


In [15]:
cnn.save('Shape_ep20_simplecnn')

INFO:tensorflow:Assets written to: Shape_ep20_simplecnn/assets


INFO:tensorflow:Assets written to: Shape_ep20_simplecnn/assets
